# Project I - Image Classification

**Team**: Filip Kołodziejczyk, Jerzy Kraszewski

## Introduction

The goal of this project is to create a model that can classify images of 10 different classes. The dataset used for this project is the CINIC-10 dataset, which is a combination of CIFAR-10 and ImageNet. The dataset contains 270,000 images, which are divided into 10 classes. The classes are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. The images are 32x32 pixels in size and are in RGB format. Data is divided into training, validation, and test sets, equally for each class.
More details about the dataset can be found [here](https://datashare.ed.ac.uk/handle/10283/3192) and [here](https://www.kaggle.com/datasets/mengcius/cinic10/data).

NOTE: To allow for different split of the dataset, all directories were merged and split is done in the code.

TODO: Add citation for this dataset

## Environment setup

In [1]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

import utils

## Extracting the data

In [ ]:
archive_path = 'data/cinic10.zip'
data_dir = 'data/'
with ZipFile(archive_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

## Loading the data

In [4]:
batch_size = 32

transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.47889522, 0.47227842, 0.43047404), (0.24205776, 0.23828046, 0.25874835)
        ), # cinic10 mean and std
    ]
)

train, test, valid = utils.split_dataset_classwise(data_dir, transforms)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid, batch_size=batch_size, shuffle=False)

## Defining the VGG model

In [5]:
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/Users/kolodziejczykf/Developer/PW/deep-learning/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kolodziejczykf/Developer/PW/deep-learning/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/kolodziejczykf/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100.0%


## Training the model

In [6]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}')

KeyboardInterrupt: 

## Model evaluation

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total:.4f}')